# Deep Learning Programming Exercise 7: Convolutional Networks in Keras

Welcome to the 7th assignment of deep learning programming!
In this assignment you will implement a convolutional neural network in Keras.

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import keras
import h5py
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import *
from utils.data_utils import load_CIFAR10
import numpy as np

Using TensorFlow backend.


## Load data

We will use the same dataset as previous assignments. 
However, note the CNN input is different from that of traditional neural networks, it is width x height x channels.

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = '../../data/cifar'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Normalize the data
    X_train /= 255
    X_val /= 255
    X_test /= 255
    X_dev /= 255
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000, 10)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000, 10)
Test data shape:  (1000, 32, 32, 3)
Test labels shape:  (1000, 10)


## Step 1: build a model [10pt]

This is an open assgiment, you should use whatever you have learnt in this course to build a promising model. For example, dropout, batch normalization, etc. After building model, you can call `model.summary()` to check architecture of your model

In [4]:
input_shape = (32, 32, 3)

### START CODE HERE ###
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same',activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
### END CODE HERE ###

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       512       
__________

## Step 2: compile, train and test your model [2pt]

After a number of epoches, you can achieve a much better performance than previous assginments. 
For example, our implementation has nearly 85% on valiation set and around 83% on test set.

Hint: use `Callbacks` in Keras to keep the best model.

Important: Store your model as model.hdf5 and upload it to the git repository.

In [5]:
### START CODE HERE ###

# save the best model only
filepath ='model.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                             monitor='val_acc', 
                                             verbose=1, 
                                             save_best_only=True, 
                                             mode='max')
callbacks_list = [checkpoint]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=SGD(lr=0.1, momentum=0, decay=0, nesterov=False),
              metrics=['accuracy'])


batch_size = 256
epochs = 50
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(X_val, y_val))


model.load_weights(filepath)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### END CODE HERE ###

Train on 49000 samples, validate on 1000 samples
Epoch 1/50
49000/49000 [==============================] - 8s 170us/step - loss: 1.7483 - acc: 0.3994 - val_loss: 1.6044 - val_acc: 0.4630

Epoch 00001: val_acc improved from -inf to 0.46300, saving model to model.hdf5
Epoch 2/50
49000/49000 [==============================] - 8s 158us/step - loss: 1.2809 - acc: 0.5400 - val_loss: 1.6196 - val_acc: 0.4510

Epoch 00002: val_acc did not improve from 0.46300
Epoch 3/50
49000/49000 [==============================] - 8s 159us/step - loss: 1.1039 - acc: 0.6060 - val_loss: 1.1159 - val_acc: 0.6130

Epoch 00003: val_acc improved from 0.46300 to 0.61300, saving model to model.hdf5
Epoch 4/50
49000/49000 [==============================] - 8s 158us/step - loss: 0.9909 - acc: 0.6494 - val_loss: 1.0744 - val_acc: 0.6090

Epoch 00004: val_acc did not improve from 0.61300
Epoch 5/50
49000/49000 [==============================] - 8s 159us/step - loss: 0.9089 - acc: 0.6783 - val_loss: 0.8943 - val_acc: 0.6

49000/49000 [==============================] - 8s 159us/step - loss: 0.2883 - acc: 0.8966 - val_loss: 0.5955 - val_acc: 0.8130

Epoch 00042: val_acc did not improve from 0.84900
Epoch 43/50
49000/49000 [==============================] - 8s 159us/step - loss: 0.2851 - acc: 0.8975 - val_loss: 0.5202 - val_acc: 0.8410

Epoch 00043: val_acc did not improve from 0.84900
Epoch 44/50
49000/49000 [==============================] - 8s 158us/step - loss: 0.2778 - acc: 0.9001 - val_loss: 0.5652 - val_acc: 0.8300

Epoch 00044: val_acc did not improve from 0.84900
Epoch 45/50
49000/49000 [==============================] - 8s 159us/step - loss: 0.2739 - acc: 0.9018 - val_loss: 0.5605 - val_acc: 0.8310

Epoch 00045: val_acc did not improve from 0.84900
Epoch 46/50
49000/49000 [==============================] - 8s 159us/step - loss: 0.2656 - acc: 0.9047 - val_loss: 0.7617 - val_acc: 0.7850

Epoch 00046: val_acc did not improve from 0.84900
Epoch 47/50
49000/49000 [==============================] - 8s 

## Model Description [3pt]

Describe in just a few words what the inuition behind the layers types in your model is, why/how they improve the performance compared to vanilla cnns. Be specific about the performance metric: Improving accuracy is not the same as improving training speed. Be critical, point out drawbacks that you can think of.

Write your answer here!